# 1. Compute Results Overview Tables

The success of an assembly is evaluated by the computation of metrics in two defined ways: globally through statistics inherent to the complete set of sequences that were assembled, and relative to the replicons present in the sample. 

The computation of the global metrics is done through custom python code from the complete set of sequences assembled. 

The following metrics are computed for the complete and filtered set of assembled sequences, restricted to contigs of length above a specified minimum size: 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Maximum sequence length:** The length of the largest contig in the assembly.
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **Contiguity**
    - **Nx (where 0  < x  ⩽ 100):** Length for which the collection of all assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the assembly
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event

After filtering, the sequences are mapped with the reference replicons and the metrics are computed through custom python code. The metrics are calculated, filtered for a minimum length, for each replicon in the file provided input references. 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **COMPASS**
    - **(Breadth of) Coverage:** Ratio of covered sequence on the reference by aligned contigs;
    - **Multiplicity:** Ratio of the length of alignable assembled sequence to covered sequence on the reference;
    - **Validity:** Ratio of the length of the alignable assembled sequence to total basepairs in the aligned contigs;
    - **Parsimony:** Cost of the assembly (multiplicity over validity);
- **Contiguity**
    - **Contiguity:** longest single alignment between the assembly and the reference, relative to the reference length;
    - **NAx (where 0  < x  ⩽ 100):** Length for which the collection of aligned assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the reference replicon;
    - **NGx (where 0  < x  ⩽ 100):** Length for which the collection of aligned contigs of that length or longer covers at least a given percentage of the sequence of the reference.
    - **Lx (where 0  < x  ⩽ 100):** Minimal number of contigs that cover x % of the sequence of the reference;
- **Identity**
    - **Identity:** Ratio of identical basepairs in all aligned contigs to the reference;
    - **Lowest identity:** Identity of the lowest scoring contig to the reference.
    - **PLS**: Phred-like score per contig, per assembler.
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event


## Imports

In [1]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np

## Global variables

In [74]:
ASSEMBLER_PROCESS_LIST = ["ABYSS", "BCALM2", "GATBMINIAPIPELINE", "METAHIPMER2", "MINIA", "MEGAHIT", "METASPADES", "UNICYCLER", "SPADES",
                          "SKESA", "VELVETOPTIMIZER", "IDBA"]
PROCESS_TO_NAME = {"ABYSS": "AbYSS",
                   "BCALM2": "BCALM2", 
                   "GATBMINIAPIPELINE": "GATBMiniaPipeline",
                   "METAHIPMER2": "MetaHipMer2",
                   "MINIA": "MINIA",
                   "MEGAHIT": "MEGAHIT", 
                   "METASPADES": "metaSPAdes", 
                   "UNICYCLER": "Unicycler", 
                   "SPADES": "SPAdes",
                    "SKESA": "SKESA",
                   "VELVETOPTIMIZER": "VelvetOptimizer",
                   "IDBA": "IDBA-UD"}

log_distributed = ['LHS', 'LNN', 'ERR2935805']
even_distribution = ['EMS', 'LNN', 'ERR2984773']

genomic_assemblers = ['AbYSS', 'BCALM2', 'MINIA', 'SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer']
metagenomic_assemblers = ['MetaHipMer2','GATBMiniaPipeline', 'IDBA-UD', 'MEGAHIT', 'metaSPAdes']
single_kmer = ['BCALM2', 'MINIA', 'AbYSS']
multiple_kmer = ['SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer', 'GATBMiniaPipeline', 
                 'IDBA-UD', 'MEGAHIT', 'metaSPAdes', 'MetaHipMer2']

## Global metrics

### Load data

In [63]:
report_glob = glob.glob('../Results/*/*/report/pipeline_report_tables.json')
global_pipeline_metrics_df = pd.DataFrame()

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for line in json_report[sample]['GlobalTable']:
                assembler = line['assembler']
                global_pipeline_metrics_df = global_pipeline_metrics_df.append({'run': stats_run,
                                                                                'sample': sample,
                                                                                'assembler': line['assembler'],
                                                                                'contigs': int(line['original']['contigs']),
                                                                                'basepairs': int(line['original']['basepairs']),
                                                                                'max_contig': int(line['original']['max_contig_size']),
                                                                                'n50': int(line['original']['N50']),
                                                                                'mapped_reads': line['original']['mapped_reads'],
                                                                                'Ns': int(line['original']['Ns']),
                                                                                'misassembled contigs': line['filtered']['misassembled_contigs'],
                                                                                'misassembly events': line['filtered']['misassembly_events'],
                                                                                'filtered_contigs': line['filtered']['contigs'],
                                                                                'filtered_basepairs': line['filtered']['basepairs'],
                                                                                'filtered_n50': line['filtered']['N50'],
                                                                                'filtered_Ns': line['filtered']['Ns'],
                                                                                'filtered_mapped_reads': line['filtered']['mapped_reads'],
                                                                               },
                                                                               ignore_index=True)
global_pipeline_metrics_df['distribution'] = np.where(global_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
global_pipeline_metrics_df['type'] = np.where(global_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
global_pipeline_metrics_df['algorythm'] = np.where(global_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50', 'filtered_n50','misassembled contigs', 'misassembly events']] = global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50','filtered_n50','misassembled contigs', 'misassembly events']].apply(pd.to_numeric)

Processing pipeline_report_tables.json data from run1...
Processing pipeline_report_tables.json data from run2...
Processing pipeline_report_tables.json data from run3...


### Preview data

In [64]:
global_pipeline_metrics_df.dtypes

Ns                       float64
assembler                 object
basepairs                float64
contigs                  float64
filtered_Ns              float64
filtered_basepairs       float64
filtered_contigs         float64
filtered_mapped_reads    float64
filtered_n50             float64
mapped_reads             float64
max_contig               float64
misassembled contigs     float64
misassembly events       float64
n50                      float64
run                       object
sample                    object
distribution              object
type                      object
algorythm                 object
dtype: object

In [65]:
global_pipeline_metrics_df

,Ns,assembler,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50,run,sample,distribution,type,algorythm
0,147.0,ABySS,91720949.0,497783.0,5.0,30657287.0,1196.0,82.019612,52869.0,92.343671,298093.0,2.0,6.0,139.0,run1,ERR2984773,Even,Metagenomic,Multiple k-mer De Bruijn graph
1,0.0,BCALM2,170402348.0,3508016.0,0.0,0.0,0.0,0.000000,0.0,8.151058,994.0,0.0,0.0,49.0,run1,ERR2984773,Even,Genomic,Single k-mer De Bruijn graph
2,0.0,GATBMiniaPipeline,32809112.0,5397.0,0.0,30564302.0,761.0,82.840039,113162.0,84.068286,768462.0,1.0,2.0,102115.0,run1,ERR2984773,Even,Metagenomic,Multiple k-mer De Bruijn graph
3,0.0,IDBA-UD,39773020.0,20164.0,0.0,31562690.0,2654.0,81.596739,29134.0,84.336773,303329.0,4.0,7.0,20993.0,run1,ERR2984773,Even,Metagenomic,Multiple k-mer De Bruijn graph
4,0.0,MEGAHIT,41166407.0,20614.0,0.0,32021766.0,1447.0,84.138912,149841.0,86.166393,1205839.0,15.0,32.0,88308.0,run1,ERR2984773,Even,Metagenomic,Multiple k-mer De Bruijn graph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,0.0,MINIA,41187149.0,67998.0,0.0,28276844.0,3973.0,74.390822,15912.0,80.663396,117945.0,0.0,0.0,7106.0,run3,ERR2984773,Even,Genomic,Single k-mer De Bruijn graph
102,0.0,SKESA,30430077.0,1352.0,0.0,30211740.0,846.0,80.664130,77870.0,81.479714,446163.0,0.0,0.0,76924.0,run3,ERR2984773,Even,Genomic,Multiple k-mer De Bruijn graph
103,0.0,SPAdes,46535940.0,39819.0,0.0,32500826.0,2049.0,82.045580,99299.0,85.446436,745370.0,9.0,18.0,53292.0,run3,ERR2984773,Even,Genomic,Multiple k-mer De Bruijn graph
104,0.0,Unicycler,30592960.0,895.0,0.0,30432114.0,425.0,81.941639,191424.0,82.823086,882182.0,2.0,4.0,191424.0,run3,ERR2984773,Even,Genomic,Multiple k-mer De Bruijn graph


### Save data

In [66]:
global_pipeline_metrics_df.to_csv("Tables/Results/Global metrics.csv")

### Global statistics per assembler type

In [67]:
stats_per_assembler_type = pd.DataFrame()
for assembly_type in global_pipeline_metrics_df.type.unique():
    row={'Assembler Type': assembly_type}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.type == assembly_type].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2), int(minimum), int(maximum))
    stats_per_assembler_type=stats_per_assembler_type.append(row, ignore_index=True)
stats_per_assembler_type = stats_per_assembler_type.set_index('Assembler Type')
stats_per_assembler_type.to_csv("Tables/Results/Global metrics per assembler type.csv")

In [68]:
stats_per_assembler_type

,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler Type,,,,,,,,,,,,,
Metagenomic,899.21 [0;8788],34000575.0 [13174826;91720949],45733.13 [640;595080],767.56 [0;8283],27525314.28 [13883;32021766],1171.89 [13;3035],86.1 [0;107],100984.79 [1039;238611],88.84 [2;108],643814.77 [1157;1205839],5.05 [0;18],9.62 [0;36],91833.59 [88;238611]
Genomic,0.0 [0;0],42206039.13 [30110758;170402348],262526.38 [656;3508016],0.0 [0;0],26882765.51 [0;32500826],1902.33 [0;6250],80.88 [0;99],95630.53 [0;245050],85.91 [8;100],522015.4 [994;1303875],2.2 [0;10],4.33 [0;19],90605.67 [49;245050]


### Global stats per assembly algorithm

In [69]:
stats_per_assembler_algo = pd.DataFrame()
for assembly_algo in global_pipeline_metrics_df.algorythm.unique():
    row={'Assembler Algorithm': assembly_algo}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.algorythm == assembly_algo].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2), int(minimum), int(maximum))
    stats_per_assembler_algo=stats_per_assembler_algo.append(row, ignore_index=True)
stats_per_assembler_algo = stats_per_assembler_algo.set_index('Assembler Algorithm')
stats_per_assembler_algo.to_csv("Tables/Results/Global metrics per assembler algorithm.csv")

In [70]:
stats_per_assembler_algo

,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler Algorithm,,,,,,,,,,,,,
Multiple k-mer De Bruijn graph,623.32 [0;8788],33441027.89 [13174826;91720949],33440.59 [640;595080],532.06 [0;8283],28419512.4 [13883;32500826],1070.4 [13;3035],88.11 [0;107],116138.57 [1039;245050],90.38 [2;108],691320.52 [1157;1303875],4.62 [0;18],8.89 [0;36],107910.89 [88;245050]
Single k-mer De Bruijn graph,0.0 [0;0],57249799.0 [30641333;170402348],647813.11 [16570;3508016],0.0 [0;0],21547307.11 [0;28279162],3494.17 [0;6250],63.2 [0;92],13514.0 [0;20782],73.97 [8;97],107066.0 [994;161812],0.0 [0;0],0.0 [0;0],10163.67 [49;17967]


### Global stats per assembler

In [71]:
stats_per_assembler = pd.DataFrame()
for assembler in global_pipeline_metrics_df.assembler.unique():
    row={'Assembler': assembler, 
         'Type':global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].type.unique()[0],
         'Algorithm': global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2), int(minimum), int(maximum))
    stats_per_assembler=stats_per_assembler.append(row, ignore_index=True)
stats_per_assembler = stats_per_assembler.set_index('Assembler')
stats_per_assembler.to_csv("Tables/Results/Global metrics per assembler.csv")

In [72]:
stats_per_assembler

,Algorithm,Ns,Type,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,2062.43 [147;3884],Metagenomic,41418778.14 [31502565;91720949],87055.86 [4770;497783],1260.43 [5;2328],30833848.86 [30657287;30988173],1165.43 [497;1825],102.83 [82;107],105261.86 [52869;138706],104.9 [92;108],811320.71 [298093;952618],4.43 [2;8],7.86 [3;14],90370.86 [139;124246]
BCALM2,Single k-mer De Bruijn graph,0.0 [0;0],Genomic,80340494.0 [31047581;170402348],1261858.33 [25377;3508016],0.0 [0;0],14860681.0 [0;28174748],3275.0 [0;6250],39.88 [0;92],7918.67 [0;20700],56.66 [8;96],66942.33 [994;161812],0.0 [0;0],0.0 [0;0],6029.67 [49;17395]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,31253502.89 [30440762;32811080],2480.89 [943;5402],0.0 [0;0],30393672.89 [30273320;30564398],744.0 [657;814],93.59 [82;98],108898.33 [90988;122545],94.75 [84;100],723087.33 [632215;768585],1.67 [1;3],3.0 [1;6],104806.0 [90346;121957]
IDBA-UD,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,33525481.22 [30390357;39779451],7944.11 [1821;20192],0.0 [0;0],30612932.33 [30129709;31562690],1728.89 [1254;2654],92.47 [81;98],52473.33 [29134;65108],94.54 [84;100],452420.67 [303329;585610],11.44 [4;18],21.22 [7;36],48807.33 [20993;63725]
MEGAHIT,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,34103641.33 [30545419;41166407],7469.0 [814;20614],0.0 [0;0],30953572.33 [30418624;32021766],830.0 [518;1447],94.89 [84;100],167083.67 [149841;179870],96.22 [86;101],868917.67 [632274;1205839],11.33 [9;15],23.56 [18;32],146076.33 [88308;178381]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,24645723.78 [13174826;30381792],7040.22 [881;19361],0.0 [0;0],20709046.78 [1597681;30264953],912.67 [712;1309],66.63 [3;98],70222.0 [1175;106637],74.44 [25;99],313232.67 [2964;468375],0.0 [0;0],0.0 [0;0],69239.33 [658;106383]
metaSPAdes,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,36633480.0 [30453778;48964620],21015.0 [640;61644],0.0 [0;0],30917818.67 [30383186;31967753],804.33 [359;1689],94.17 [83;100],180893.67 [95101;238611],95.36 [85;100],809315.0 [473277;1036942],1.67 [1;2],4.0 [3;6],163304.0 [42332;238611]
MINIA,Single k-mer De Bruijn graph,0.0 [0;0],Genomic,34159104.0 [30641333;41187149],33767.89 [16570;67998],0.0 [0;0],28233933.22 [28205937;28279162],3713.33 [3574;3980],86.52 [74;92],19109.33 [15912;20782],91.29 [80;97],147189.67 [117945;161812],0.0 [0;0],0.0 [0;0],14297.67 [7106;17967]
SKESA,Multiple k-mer De Bruijn graph,0.0 [0;0],Genomic,30263322.33 [30110758;30430077],1647.67 [1269;2322],0.0 [0;0],30037221.0 [29835112;30211740],1154.33 [846;1726],91.9 [80;97],65058.67 [43168;77870],92.76 [81;98],569520.0 [446163;632013],6.33 [0;10],12.33 [0;19],64292.67 [42115;76924]


### Global stats per distribution

In [73]:
for distribution in global_pipeline_metrics_df.distribution.unique():
    stats_per_distribution = pd.DataFrame()
    for assembler in global_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':global_pipeline_metrics_df[(global_pipeline_metrics_df.assembler == assembler) & (global_pipeline_metrics_df.distribution == distribution)].type.unique()[0],
             'Algorithm': global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=global_pipeline_metrics_df[(global_pipeline_metrics_df.assembler == assembler) & (global_pipeline_metrics_df.distribution == distribution)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            minimum = lala.loc['min',column]
            maximum = lala.loc['max',column]
            row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
        stats_per_distribution=stats_per_distribution.append(row, ignore_index=True)
    stats_per_distribution = stats_per_distribution.set_index('Assembler')
    stats_per_distribution.to_csv("Tables/Results/{} - Global metrics per assembler.csv".format(distribution))
    print(distribution)
    display(stats_per_distribution)

Even


,Algorithm,Ns,Type,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,2062.43 [147;3884],Metagenomic,41418778.14 [31502565;91720949],87055.86 [4770;497783],1260.43 [5;2328],30833848.86 [30657287;30988173],1165.43 [497;1825],102.83 [82;107],105261.86 [52869;138706],104.9 [92;108],811320.71 [298093;952618],4.43 [2;8],7.86 [3;14],90370.86 [139;124246]
BCALM2,Single k-mer De Bruijn graph,0.0 [0;0],Genomic,80340494.0 [31047581;170402348],1261858.33 [25377;3508016],0.0 [0;0],14860681.0 [0;28174748],3275.0 [0;6250],39.88 [0;92],7918.67 [0;20700],56.66 [8;96],66942.33 [994;161812],0.0 [0;0],0.0 [0;0],6029.67 [49;17395]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,31253502.89 [30440762;32811080],2480.89 [943;5402],0.0 [0;0],30393672.89 [30273320;30564398],744.0 [657;814],93.59 [82;98],108898.33 [90988;122545],94.75 [84;100],723087.33 [632215;768585],1.67 [1;3],3.0 [1;6],104806.0 [90346;121957]
IDBA-UD,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,33525481.22 [30390357;39779451],7944.11 [1821;20192],0.0 [0;0],30612932.33 [30129709;31562690],1728.89 [1254;2654],92.47 [81;98],52473.33 [29134;65108],94.54 [84;100],452420.67 [303329;585610],11.44 [4;18],21.22 [7;36],48807.33 [20993;63725]
MEGAHIT,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,34103641.33 [30545419;41166407],7469.0 [814;20614],0.0 [0;0],30953572.33 [30418624;32021766],830.0 [518;1447],94.89 [84;100],167083.67 [149841;179870],96.22 [86;101],868917.67 [632274;1205839],11.33 [9;15],23.56 [18;32],146076.33 [88308;178381]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,24645723.78 [13174826;30381792],7040.22 [881;19361],0.0 [0;0],20709046.78 [1597681;30264953],912.67 [712;1309],66.63 [3;98],70222.0 [1175;106637],74.44 [25;99],313232.67 [2964;468375],0.0 [0;0],0.0 [0;0],69239.33 [658;106383]
metaSPAdes,Multiple k-mer De Bruijn graph,0.0 [0;0],Metagenomic,36633480.0 [30453778;48964620],21015.0 [640;61644],0.0 [0;0],30917818.67 [30383186;31967753],804.33 [359;1689],94.17 [83;100],180893.67 [95101;238611],95.36 [85;100],809315.0 [473277;1036942],1.67 [1;2],4.0 [3;6],163304.0 [42332;238611]
MINIA,Single k-mer De Bruijn graph,0.0 [0;0],Genomic,34159104.0 [30641333;41187149],33767.89 [16570;67998],0.0 [0;0],28233933.22 [28205937;28279162],3713.33 [3574;3980],86.52 [74;92],19109.33 [15912;20782],91.29 [80;97],147189.67 [117945;161812],0.0 [0;0],0.0 [0;0],14297.67 [7106;17967]
SKESA,Multiple k-mer De Bruijn graph,0.0 [0;0],Genomic,30263322.33 [30110758;30430077],1647.67 [1269;2322],0.0 [0;0],30037221.0 [29835112;30211740],1154.33 [846;1726],91.9 [80;97],65058.67 [43168;77870],92.76 [81;98],569520.0 [446163;632013],6.33 [0;10],12.33 [0;19],64292.67 [42115;76924]


## Reference-based metrics

### Load data

In [78]:
report_glob = glob.glob('../Results/*/*/report/pipeline_report_tables.json')
reference_pipeline_metrics_df = pd.DataFrame()

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for reference, data in json_report[sample]['ReferenceTables'].items():
                for row in data:
                    for item in row:
                        reference_pipeline_metrics_df = reference_pipeline_metrics_df.append({'run': stats_run,
                                                                                        'sample': sample,
                                                                                        'assembler': item['assembler'],
                                                                                        'reference': reference,
                                                                                        'LSA': item['contiguity'],
                                                                                        'breadth_of_coverage': item['breadth_of_coverage'],
                                                                                        'multiplicity': item['multiplicity'],
                                                                                        'validity': item['validity'],
                                                                                        'parsimony': item['parsimony'],
                                                                                        'identity': item['identity'],
                                                                                        'lowest_identity': item['lowest_identity'],
                                                                                        'L90': item['L90'],
                                                                                        'contigs': item['aligned_contigs'],
                                                                                        'NA50': item['NA50'],
                                                                                        'NG50': item['NG50'],
                                                                                        'basepairs': item['aligned_basepairs'],
                                                                                        'Ns': item['Ns'],
                                                                                        'misassembled contigs': item['misassembled_contigs'],
                                                                                        'misassembly events': item['misassembly_events']},
                                                                                       ignore_index=True)

reference_pipeline_metrics_df['distribution'] = np.where(reference_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
reference_pipeline_metrics_df['type'] = np.where(reference_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
reference_pipeline_metrics_df['algorythm'] = np.where(reference_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events', 'multiplicity','validity','parsimony','identity','lowest_identity']] = reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events','multiplicity','validity','parsimony','identity','lowest_identity']].apply(pd.to_numeric)
display(reference_pipeline_metrics_df)

Processing pipeline_report_tables.json data from run1...
Processing pipeline_report_tables.json data from run2...
Processing pipeline_report_tables.json data from run3...


,L90,LSA,NA50,NG50,Ns,assembler,basepairs,breadth_of_coverage,contigs,identity,...,misassembly events,multiplicity,parsimony,reference,run,sample,validity,distribution,type,algorythm
0,39.0,0.073679,98440.0,98440.0,0.0,ABySS,3977886.0,0.983244,66.0,0.999639,...,0.0,1.001060,1.001159,Bacillus_subtilis,run1,ERR2984773,0.999901,Even,Metagenomic,Multiple k-mer De Bruijn graph
1,0.0,0.000000,0.0,0.0,0.0,BCALM2,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,Bacillus_subtilis,run1,ERR2984773,0.000000,Even,Genomic,Single k-mer De Bruijn graph
2,15.0,0.189942,263420.0,257358.0,0.0,GATBMiniaPipeline,3982010.0,0.984263,30.0,0.999809,...,0.0,1.000079,1.000083,Bacillus_subtilis,run1,ERR2984773,0.999995,Even,Metagenomic,Multiple k-mer De Bruijn graph
3,120.0,0.031121,34330.0,34330.0,0.0,IDBA-UD,3976919.0,0.983005,184.0,0.999756,...,0.0,1.003298,1.003311,Bacillus_subtilis,run1,ERR2984773,0.999986,Even,Metagenomic,Multiple k-mer De Bruijn graph
4,14.0,0.103094,294549.0,294549.0,0.0,MEGAHIT,3989518.0,0.986119,31.0,0.978636,...,4.0,0.998972,1.011678,Bacillus_subtilis,run1,ERR2984773,0.987440,Even,Metagenomic,Multiple k-mer De Bruijn graph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,0.0,0.000000,0.0,0.0,0.0,MINIA,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,Staphylococcus_aureus_plasmid3,run3,ERR2984773,0.000000,Even,Genomic,Single k-mer De Bruijn graph
1268,0.0,0.929502,2782.0,2782.0,0.0,SKESA,2782.0,0.929502,1.0,1.000000,...,0.0,1.000000,1.000000,Staphylococcus_aureus_plasmid3,run3,ERR2984773,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph
1269,0.0,0.000000,0.0,0.0,0.0,SPAdes,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,Staphylococcus_aureus_plasmid3,run3,ERR2984773,0.000000,Even,Genomic,Multiple k-mer De Bruijn graph
1270,0.0,1.000000,2993.0,2993.0,0.0,Unicycler,2993.0,1.000000,1.0,1.000000,...,0.0,1.000000,1.000000,Staphylococcus_aureus_plasmid3,run3,ERR2984773,1.000000,Even,Genomic,Multiple k-mer De Bruijn graph


In [79]:
reference_pipeline_metrics_df.dtypes

L90                     float64
LSA                     float64
NA50                    float64
NG50                    float64
Ns                      float64
assembler                object
basepairs               float64
breadth_of_coverage     float64
contigs                 float64
identity                float64
lowest_identity         float64
misassembled contigs    float64
misassembly events      float64
multiplicity            float64
parsimony               float64
reference                object
run                      object
sample                   object
validity                float64
distribution             object
type                     object
algorythm                object
dtype: object

### Save data

In [80]:
reference_pipeline_metrics_df.to_csv("Tables/Results/Reference metrics.csv")

### Reference statistics per assembler type

In [81]:
ref_stats_per_assembler_type = pd.DataFrame()
for assembly_type in reference_pipeline_metrics_df.type.unique():
    row={'Assembler Type': assembly_type}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.type == assembly_type].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_assembler_type=ref_stats_per_assembler_type.append(row, ignore_index=True)
ref_stats_per_assembler_type = ref_stats_per_assembler_type.set_index('Assembler Type')
ref_stats_per_assembler_type.to_csv("Tables/Results/Reference metrics per assembler type.csv")

In [82]:
ref_stats_per_assembler_type

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler Type,,,,,,,,,,,,,,,
Metagenomic,35.26 [0;1004],0.28 [0;1],89746.41 [0;542872],89131.67 [0;542872],91.4 [0;12098],2267909.42 [0;6780808],0.86 [0;1],85.92 [0;1404],0.92 [0;1],0.89 [0;1],0.45 [0;9],0.83 [0;15],0.93 [0;1],0.93 [0;1],0.92 [0;1]
Genomic,45.86 [0;921],0.21 [0;1],76036.23 [0;475179],74709.01 [0;475179],0.0 [0;0],2223458.03 [0;6728888],0.77 [0;1],150.17 [0;1495],0.84 [0;1],0.8 [0;1],0.22 [0;7],0.39 [0;15],0.84 [0;1],0.84 [0;1],0.84 [0;1]


### Reference stats per assembly algorithm

In [83]:
ref_stats_per_assembler_algo = pd.DataFrame()
for assembly_algo in reference_pipeline_metrics_df.algorythm.unique():
    row={'Assembler Algorithm': assembly_algo}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.algorythm == assembly_algo].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_assembler_algo=ref_stats_per_assembler_algo.append(row, ignore_index=True)
ref_stats_per_assembler_algo = ref_stats_per_assembler_algo.set_index('Assembler Algorithm')
ref_stats_per_assembler_algo.to_csv("Tables/Results/Reference metrics per assembler algorithm.csv")

In [84]:
ref_stats_per_assembler_algo

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler Algorithm,,,,,,,,,,,,,,,
Multiple k-mer De Bruijn graph,34.13 [0;1004],0.29 [0;1],98617.13 [0;542872],97618.16 [0;542872],63.36 [0;12098],2342800.58 [0;6780808],0.87 [0;1],76.95 [0;1404],0.93 [0;1],0.89 [0;1],0.42 [0;9],0.78 [0;15],0.94 [0;1],0.94 [0;1],0.93 [0;1]
Single k-mer De Bruijn graph,67.3 [0;921],0.06 [0;0],12103.01 [0;65138],11585.5 [0;64960],0.0 [0;0],1790646.37 [0;6518607],0.56 [0;0],290.36 [0;1495],0.65 [0;1],0.65 [0;1],0.0 [0;0],0.0 [0;0],0.65 [0;1],0.65 [0;1],0.65 [0;1]


### Reference stats per assembler

In [85]:
ref_stats_per_assembler = pd.DataFrame()
for assembler in reference_pipeline_metrics_df.assembler.unique():
    row={'Assembler': assembler, 
         'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
         'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_assembler=ref_stats_per_assembler.append(row, ignore_index=True)
ref_stats_per_assembler = ref_stats_per_assembler.set_index('Assembler')
ref_stats_per_assembler.to_csv("Tables/Results/Reference metrics per assembler.csv")

In [86]:
ref_stats_per_assembler

,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,60.88 [0;1004],0.32 [0;0],110190.23 [0;351502],110089.38 [0;351502],105.21 [0;2170],Metagenomic,2536099.3 [0;6780808],0.92 [0;0],97.05 [0;1404],0.99 [0;1],0.95 [0;1],0.37 [0;8],0.65 [0;14],1.0 [0;1],1.0 [0;1],0.99 [0;1]
BCALM2,Single k-mer De Bruijn graph,51.2 [0;921],0.04 [0;0],6932.33 [0;65138],6539.94 [0;64934],0.0 [0;0],Genomic,1236061.67 [0;6510588],0.37 [0;0],272.97 [0;1495],0.5 [0;1],0.5 [0;1],0.0 [0;0],0.0 [0;0],0.5 [0;1],0.5 [0;1],0.5 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,31.33 [0;112],0.35 [0;1],106093.36 [2176;452364],96782.33 [2176;351186],0.0 [0;0],Metagenomic,2523831.42 [2176;6716518],0.98 [0;1],59.92 [1;199],1.0 [0;1],0.98 [0;1],0.14 [0;3],0.25 [0;6],1.0 [0;1],1.0 [0;1],1.0 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,68.82 [0;283],0.27 [0;1],55837.36 [0;254629],54555.43 [0;254629],0.0 [0;0],Metagenomic,2507606.06 [0;6710104],0.94 [0;1],121.11 [0;449],0.97 [0;1],0.87 [0;1],1.12 [0;9],1.94 [0;15],0.98 [0;1],0.98 [0;1],0.97 [0;1]
MEGAHIT,Multiple k-mer De Bruijn graph,18.31 [0;55],0.37 [0;1],138989.33 [2215;542872],141793.83 [2215;542872],0.0 [0;0],Metagenomic,2534425.75 [2215;6737986],0.99 [0;1],44.39 [1;141],1.0 [0;1],0.93 [0;1],0.97 [0;4],1.99 [0;7],1.0 [0;1],1.01 [0;1],1.0 [0;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,21.69 [0;110],0.23 [0;0],58464.84 [0;257424],57747.71 [0;257424],0.0 [0;0],Metagenomic,1722416.51 [0;6717767],0.68 [0;0],74.19 [0;361],0.94 [0;1],0.94 [0;1],0.0 [0;0],0.0 [0;0],0.94 [0;1],0.94 [0;1],0.94 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,20.14 [0;90],0.34 [0;1],133559.97 [1677;475510],132927.86 [1677;475510],0.0 [0;0],Metagenomic,2530047.11 [2076;6726884],0.98 [0;1],39.33 [1;167],1.0 [0;1],0.99 [0;1],0.14 [0;2],0.33 [0;6],1.01 [0;1],1.01 [0;1],1.0 [0;1]
MINIA,Single k-mer De Bruijn graph,83.4 [0;406],0.08 [0;0],17273.69 [0;65138],16631.06 [0;64960],0.0 [0;0],Genomic,2345231.06 [0;6518607],0.74 [0;0],307.75 [0;1146],0.81 [0;1],0.79 [0;1],0.0 [0;0],0.0 [0;0],0.81 [0;1],0.81 [0;1],0.81 [0;1]
SKESA,Multiple k-mer De Bruijn graph,57.92 [0;670],0.27 [0;0],76053.75 [1404;351014],75134.92 [0;351014],0.0 [0;0],Genomic,2496709.31 [1460;6673601],0.9 [0;0],96.06 [1;933],1.0 [0;1],0.87 [0;1],0.67 [0;7],1.17 [0;15],1.0 [0;1],1.0 [0;1],1.0 [0;1]


### Reference stats per reference replicon

#### Per assembler for each reference replicon

In [87]:
reference_pipeline_metrics_df.reference.unique()

array(['Bacillus_subtilis', 'Enterococcus_faecalis',
       'Escherichia_coli_plasmid', 'Escherichia_coli',
       'Lactobacillus_fermentum', 'Listeria_monocytogenes',
       'Pseudomonas_aeruginosa', 'Salmonella_enterica',
       'Staphylococcus_aureus', 'Staphylococcus_aureus_plasmid1',
       'Staphylococcus_aureus_plasmid2', 'Staphylococcus_aureus_plasmid3'],
      dtype=object)

In [88]:
for reference in reference_pipeline_metrics_df.reference.unique():
    ref_stats_per_reference_per_assembler = pd.DataFrame()
    for assembler in reference_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.assembler == assembler) & (reference_pipeline_metrics_df.reference == reference)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            minimum = lala.loc['min',column]
            maximum = lala.loc['max',column]
            row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
        ref_stats_per_reference_per_assembler=ref_stats_per_reference_per_assembler.append(row, ignore_index=True)
    ref_stats_per_reference_per_assembler = ref_stats_per_reference_per_assembler.set_index('Assembler')
    ref_stats_per_reference_per_assembler.to_csv("Tables/Results/{} - Reference metrics per assembler.csv".format(reference))
    print(reference)
    display(ref_stats_per_reference_per_assembler)

Bacillus_subtilis


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,18.0 [13;39],0.1 [0;0],250023.14 [98440;295233],250023.14 [98440;295233],20.14 [0;47],Metagenomic,3988493.71 [3977886;3991152],0.99 [0;0],30.0 [22;66],1.0 [0;0],1.0 [0;0],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [0;0]
BCALM2,Single k-mer De Bruijn graph,22.0 [0;66],0.01 [0;0],22400.0 [0;65138],22174.33 [0;64934],0.0 [0;0],Genomic,2265479.33 [0;3951974],0.56 [0;0],527.67 [0;1467],0.67 [0;1],0.67 [0;1],0.0 [0;0],0.0 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,14.0 [13;15],0.16 [0;0],265520.0 [263420;269695],261389.67 [257358;263445],0.0 [0;0],Metagenomic,3981896.67 [3979577;3984103],0.98 [0;0],28.33 [27;30],1.0 [0;1],1.0 [0;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,52.33 [18;120],0.07 [0;0],166650.67 [34330;252653],153422.67 [34330;212969],0.0 [0;0],Metagenomic,3977018.67 [3976292;3977845],0.98 [0;0],85.33 [34;184],1.0 [0;0],0.94 [0;0],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [0;0]
MEGAHIT,Multiple k-mer De Bruijn graph,13.67 [13;14],0.13 [0;0],274090.0 [263586;294549],274090.0 [263586;294549],0.0 [0;0],Metagenomic,3988399.89 [3986471;3989518],0.99 [0;0],28.33 [27;31],0.99 [0;0],0.79 [0;0],1.0 [0;2],2.33 [0;4],1.0 [0;0],1.0 [0;1],1.0 [0;0]
MetaHipMer2,Multiple k-mer De Bruijn graph,10.33 [0;16],0.07 [0;0],171945.22 [1145;257424],171562.67 [0;257424],0.0 [0;0],Metagenomic,2721019.89 [205828;3976387],0.67 [0;0],77.11 [27;180],1.0 [0;1],0.99 [0;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,14.67 [9;24],0.18 [0;0],272233.67 [164894;343843],272233.67 [164894;343843],0.0 [0;0],Metagenomic,3981566.67 [3975594;3987932],0.98 [0;0],30.67 [18;51],1.0 [0;0],0.98 [0;0],0.0 [0;0],0.0 [0;0],1.0 [0;1],1.0 [0;1],1.0 [0;1]
MINIA,Single k-mer De Bruijn graph,78.0 [66;102],0.03 [0;0],58810.33 [46155;65138],57658.67 [43056;64960],0.0 [0;0],Genomic,3951436.44 [3949080;3953200],0.98 [0;0],134.11 [116;170],1.0 [0;1],1.0 [0;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [0;1]
SKESA,Multiple k-mer De Bruijn graph,17.0 [14;20],0.1 [0;0],242427.67 [200881;269493],240318.0 [200881;263164],0.0 [0;0],Genomic,3969836.33 [3967464;3971432],0.98 [0;0],30.0 [26;34],1.0 [0;1],1.0 [0;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1]


Enterococcus_faecalis


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,17.29 [9;61],0.17 [0;0],212767.0 [45769;241090],212767.0 [45769;241090],0.0 [0;0],Metagenomic,2823330.57 [2813974;2828460],0.99 [0;0],32.71 [19;106],1.0 [0;0],1.0 [0;0],0.0 [0;0],0.0 [0;0],1.01 [1;1],1.01 [1;1],1.0 [0;1]
BCALM2,Single k-mer De Bruijn graph,34.78 [0;107],0.02 [0;0],10553.67 [0;30453],10140.0 [0;30452],0.0 [0;0],Genomic,993772.0 [0;2768590],0.35 [0;0],115.22 [0;180],0.67 [0;1],0.67 [0;1],0.0 [0;0],0.0 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,18.33 [13;24],0.17 [0;0],180947.33 [127741;235254],180947.33 [127741;235254],0.0 [0;0],Metagenomic,2798634.44 [2779793;2820351],0.98 [0;0],37.44 [28;46],1.0 [0;1],0.99 [0;1],0.33 [0;1],0.67 [0;2],1.0 [1;1],1.0 [1;1],1.0 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,65.67 [22;147],0.1 [0;0],81575.67 [18985;113206],83696.0 [18985;118897],0.0 [0;0],Metagenomic,2799260.67 [2777299;2811602],0.98 [0;0],113.0 [53;231],1.0 [0;0],0.93 [0;0],1.0 [0;3],2.0 [0;6],1.0 [0;1],1.0 [1;1],1.0 [0;0]
MEGAHIT,Multiple k-mer De Bruijn graph,14.33 [13;17],0.23 [0;0],235337.67 [235316;235381],236414.33 [235316;238546],0.0 [0;0],Metagenomic,2802086.33 [2785847;2822833],0.98 [0;0],32.44 [31;34],1.0 [0;0],0.96 [0;0],0.67 [0;1],1.33 [0;2],1.0 [0;1],1.0 [1;1],1.0 [0;0]
MetaHipMer2,Multiple k-mer De Bruijn graph,17.89 [0;27],0.09 [0;0],79651.22 [1159;118895],77900.67 [0;118895],0.0 [0;0],Metagenomic,1936605.78 [207594;2811406],0.68 [0;0],88.67 [45;177],1.0 [0;1],1.0 [0;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1]
metaSPAdes,Multiple k-mer De Bruijn graph,15.0 [9;25],0.17 [0;0],206754.33 [143041;238611],206754.33 [143041;238611],0.0 [0;0],Metagenomic,2806275.33 [2786440;2818677],0.99 [0;0],31.67 [21;51],1.0 [0;0],0.98 [0;0],0.33 [0;1],0.67 [0;2],1.0 [1;1],1.0 [1;1],1.0 [0;1]
MINIA,Single k-mer De Bruijn graph,119.11 [103;149],0.04 [0;0],27073.11 [20287;30453],26905.44 [19908;30453],0.0 [0;0],Genomic,2749839.33 [2732359;2770364],0.97 [0;0],183.67 [166;219],1.0 [0;0],1.0 [0;0],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [0;1]
SKESA,Multiple k-mer De Bruijn graph,30.33 [19;42],0.12 [0;0],105908.0 [76714;152709],105785.67 [76714;152709],0.0 [0;0],Genomic,2801849.33 [2797241;2810284],0.98 [0;0],54.67 [40;72],0.99 [0;1],0.72 [0;1],0.67 [0;1],0.67 [0;1],1.0 [1;1],1.01 [1;1],0.99 [0;1]


Escherichia_coli_plasmid


...

,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,0.0 [0;0],0.86 [0;0],5429.57 [0;6335],5429.57 [0;6335],0.0 [0;0],Metagenomic,5429.57 [0;6335],0.86 [0;0],0.86 [0;1],0.86 [0;1],0.86 [0;1],0.0 [0;0],0.0 [0;0],0.86 [0;1],0.86 [0;1],0.86 [0;1]
BCALM2,Single k-mer De Bruijn graph,0.0 [0;0],0.33 [0;0],2112.0 [0;6336],2112.0 [0;6336],0.0 [0;0],Genomic,2112.0 [0;6336],0.33 [0;0],0.33 [0;1],0.33 [0;1],0.33 [0;1],0.0 [0;0],0.0 [0;0],0.33 [0;1],0.33 [0;1],0.33 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0 [0;0],1.01 [0;1],6394.67 [6334;6515],6394.67 [6334;6515],0.0 [0;0],Metagenomic,6335.67 [6334;6337],1.0 [0;1],1.0 [1;1],1.0 [0;1],1.0 [0;1],0.0 [0;0],0.0 [0;0],1.01 [1;1],1.01 [1;1],1.0 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,0.0 [0;0],0.92 [0;0],5812.33 [4793;6336],5812.33 [4793;6336],0.0 [0;0],Metagenomic,5802.33 [4763;6336],0.92 [0;0],1.0 [1;1],1.0 [0;1],1.0 [0;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1]
MEGAHIT,Multiple k-mer De Bruijn graph,0.0 [0;0],1.0 [0;1],6479.67 [6308;6795],6479.67 [6308;6795],0.0 [0;0],Metagenomic,6327.0 [6308;6337],1.0 [0;1],1.0 [1;1],0.98 [0;1],0.98 [0;1],0.0 [0;0],0.0 [0;0],1.01 [1;1],1.02 [1;1],0.98 [0;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],0.67 [0;0],4222.67 [0;6334],4222.67 [0;6334],0.0 [0;0],Metagenomic,4222.67 [0;6334],0.67 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1],0.0 [0;0],0.0 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,0.33 [0;1],0.89 [0;0],5636.33 [4237;6336],5636.33 [4237;6336],0.0 [0;0],Metagenomic,6292.0 [6204;6336],0.99 [0;0],1.33 [1;2],1.0 [0;1],1.0 [0;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1]
MINIA,Single k-mer De Bruijn graph,0.0 [0;0],0.67 [0;0],4214.33 [0;6336],4214.33 [0;6336],0.0 [0;0],Genomic,4214.33 [0;6336],0.67 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1],0.0 [0;0],0.0 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1]
SKESA,Multiple k-mer De Bruijn graph,0.0 [0;0],0.72 [0;0],4561.0 [1672;6044],4561.0 [1672;6044],0.0 [0;0],Genomic,5624.67 [4863;6044],0.89 [0;0],1.67 [1;3],1.0 [0;1],0.99 [0;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [0;1]


Staphylococcus_aureus_plasmid2


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,0.0 [0;0],0.95 [0;0],2094.57 [1378;2215],2094.57 [1378;2215],0.0 [0;0],Metagenomic,2094.57 [1378;2215],0.95 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1]
BCALM2,Single k-mer De Bruijn graph,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],Genomic,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0 [0;0],1.02 [0;1],2255.11 [2176;2389],2255.11 [2176;2389],0.0 [0;0],Metagenomic,2202.33 [2176;2216],0.99 [0;1],1.0 [1;1],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.02 [1;1],1.02 [1;1],1.0 [1;1]
IDBA-UD,Multiple k-mer De Bruijn graph,0.0 [0;0],0.68 [0;1],1510.0 [0;2315],1510.0 [0;2315],0.0 [0;0],Metagenomic,1477.0 [0;2216],0.67 [0;1],0.67 [0;1],0.67 [0;1],0.67 [0;1],0.0 [0;0],0.0 [0;0],0.68 [0;1],0.68 [0;1],0.67 [0;1]
MEGAHIT,Multiple k-mer De Bruijn graph,0.0 [0;0],1.02 [0;1],2262.33 [2215;2357],2262.33 [2215;2357],0.0 [0;0],Metagenomic,2215.33 [2215;2216],1.0 [0;1],1.0 [1;1],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.02 [1;1],1.02 [1;1],1.0 [1;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],0.67 [0;0],1475.33 [0;2213],1475.33 [0;2213],0.0 [0;0],Metagenomic,1475.33 [0;2213],0.67 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1],0.0 [0;0],0.0 [0;0],0.67 [0;1],0.67 [0;1],0.67 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,0.0 [0;0],0.97 [0;1],2141.0 [2076;2271],2141.0 [2076;2271],0.0 [0;0],Metagenomic,2122.67 [2076;2216],0.96 [0;1],1.0 [1;1],1.0 [0;0],1.0 [0;0],0.0 [0;0],0.0 [0;0],1.01 [1;1],1.01 [1;1],1.0 [1;1]
MINIA,Single k-mer De Bruijn graph,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],Genomic,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0]
SKESA,Multiple k-mer De Bruijn graph,0.0 [0;0],0.81 [0;0],1790.67 [1460;2005],1790.67 [1460;2005],0.0 [0;0],Genomic,1790.67 [1460;2005],0.81 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1]


Staphylococcus_aureus_plasmid3


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,0.0 [0;0],0.95 [0;0],2843.43 [1955;2992],2843.43 [1955;2992],0.0 [0;0],Metagenomic,2843.43 [1955;2992],0.95 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1]
BCALM2,Single k-mer De Bruijn graph,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],Genomic,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0 [0;0],0.99 [0;1],2975.89 [2789;3158],2975.89 [2789;3158],0.0 [0;0],Metagenomic,2924.67 [2789;2993],0.98 [0;1],1.0 [1;1],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.02 [1;1],1.02 [1;1],1.0 [1;1]
IDBA-UD,Multiple k-mer De Bruijn graph,0.33 [0;1],0.82 [0;0],2460.67 [1688;2847],2460.67 [1688;2847],0.0 [0;0],Metagenomic,2817.0 [2757;2847],0.94 [0;0],1.33 [1;2],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.01 [1;1],1.01 [1;1],1.0 [1;1]
MEGAHIT,Multiple k-mer De Bruijn graph,0.0 [0;0],1.02 [0;1],3039.33 [2992;3134],3039.33 [2992;3134],0.0 [0;0],Metagenomic,2992.33 [2992;2993],1.0 [0;1],1.0 [1;1],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.02 [1;1],1.02 [1;1],1.0 [1;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,0.0 [0;0],0.84 [0;0],2524.33 [1593;2990],2524.33 [1593;2990],0.0 [0;0],Metagenomic,2524.33 [1593;2990],0.84 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1]
metaSPAdes,Multiple k-mer De Bruijn graph,0.67 [0;1],0.8 [0;1],2403.67 [1677;3857],2403.67 [1677;3857],0.0 [0;0],Metagenomic,2944.0 [2847;2993],0.98 [0;1],1.67 [1;2],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.11 [1;1],1.11 [1;1],1.0 [1;1]
MINIA,Single k-mer De Bruijn graph,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],Genomic,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0]
SKESA,Multiple k-mer De Bruijn graph,0.0 [0;0],0.85 [0;0],2536.67 [2110;2782],2536.67 [2110;2782],0.0 [0;0],Genomic,2536.67 [2110;2782],0.85 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1],0.0 [0;0],0.0 [0;0],1.0 [1;1],1.0 [1;1],1.0 [1;1]


#### Per reference replicon for all assemblers - Even only!

In [89]:
ref_stats_per_reference_all_assemblers = pd.DataFrame()
for reference in reference_pipeline_metrics_df.reference.unique():
    row={'Reference': reference}
    lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.reference == reference) & (reference_pipeline_metrics_df.distribution == 'Even')].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_reference_all_assemblers=ref_stats_per_reference_all_assemblers.append(row, ignore_index=True)
ref_stats_per_reference_all_assemblers = ref_stats_per_reference_all_assemblers.set_index('Reference')
ref_stats_per_reference_all_assemblers.to_csv("Tables/Results/Reference metrics per replicon.csv")
display(ref_stats_per_reference_all_assemblers)

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Reference,,,,,,,,,,,,,,,
Bacillus_subtilis,25.53 [0;120],0.1 [0;0],197321.07 [0;343843],195383.53 [0;343843],1.33 [0;47],3611013.57 [0;3991152],0.89 [0;0],92.08 [0;1467],0.94 [0;1],0.92 [0;1],0.08 [0;2],0.2 [0;4],0.94 [0;1],0.94 [0;1],0.94 [0;1]
Enterococcus_faecalis,36.05 [0;149],0.13 [0;0],135017.9 [0;377104],133464.98 [0;377104],0.0 [0;0],2488219.36 [0;2828460],0.87 [0;0],72.08 [0;231],0.94 [0;1],0.91 [0;1],0.34 [0;3],0.62 [0;6],0.95 [0;1],0.95 [0;1],0.94 [0;1]
Escherichia_coli_plasmid,3.29 [0;18],0.22 [0;0],17648.73 [0;55639],17101.08 [0;55639],37.95 [0;835],82168.47 [0;109516],0.75 [0;0],9.25 [0;33],0.89 [0;1],0.88 [0;1],0.0 [0;0],0.0 [0;0],0.89 [0;1],0.89 [0;1],0.89 [0;1]
Escherichia_coli,69.48 [0;283],0.04 [0;0],51234.25 [0;138793],50949.24 [0;138793],1.27 [0;45],4046356.04 [0;4706885],0.85 [0;0],365.0 [0;1495],0.94 [0;1],0.84 [0;1],1.08 [0;8],2.01 [0;15],0.95 [0;1],0.95 [0;1],0.94 [0;1]
Lactobacillus_fermentum,77.43 [0;263],0.05 [0;0],30491.53 [0;46932],29253.78 [0;52972],0.0 [0;0],1583202.84 [0;1873053],0.83 [0;0],105.51 [0;282],0.91 [0;1],0.9 [0;1],0.34 [0;4],0.7 [0;8],0.92 [0;1],0.92 [0;1],0.92 [0;1]
Listeria_monocytogenes,24.25 [0;122],0.15 [0;0],238298.03 [0;542872],229526.71 [0;542872],0.03 [0;1],2622957.47 [0;2961845],0.88 [0;0],62.58 [0;361],0.94 [0;1],0.93 [0;1],0.03 [0;1],0.06 [0;2],0.94 [0;1],0.94 [0;1],0.94 [0;1]
Pseudomonas_aeruginosa,147.12 [0;1004],0.06 [0;0],142541.04 [0;338923],147588.8 [0;464977],588.46 [0;12098],6106074.03 [0;6780808],0.9 [0;0],239.87 [0;1404],0.94 [0;1],0.85 [0;1],1.29 [0;8],2.3 [0;15],0.95 [0;1],0.95 [0;1],0.94 [0;1]
Salmonella_enterica,53.86 [0;200],0.05 [0;0],71537.42 [0;223960],70549.1 [0;223960],0.1 [0;2],4069395.9 [0;4703394],0.85 [0;0],333.5 [0;1479],0.94 [0;1],0.83 [0;1],0.78 [0;9],1.25 [0;14],0.95 [0;1],0.95 [0;1],0.94 [0;1]
Staphylococcus_aureus,40.02 [0;140],0.12 [0;0],114659.96 [0;394879],113926.2 [0;394879],2.01 [0;71],2370560.05 [0;2693907],0.87 [0;0],76.06 [0;249],0.94 [0;1],0.91 [0;1],0.26 [0;3],0.61 [0;7],0.94 [0;1],0.94 [0;1],0.94 [0;1]


### Reference stats per distribution

In [90]:
for distribution in reference_pipeline_metrics_df.distribution.unique():
    ref_stats_per_reference_per_assembler = pd.DataFrame()
    for assembler in reference_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.assembler == assembler) & (reference_pipeline_metrics_df.distribution == distribution)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            minimum = lala.loc['min',column]
            maximum = lala.loc['max',column]
            row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
        ref_stats_per_reference_per_assembler=ref_stats_per_reference_per_assembler.append(row, ignore_index=True)
    ref_stats_per_reference_per_assembler = ref_stats_per_reference_per_assembler.set_index('Assembler')
    ref_stats_per_reference_per_assembler.to_csv("Tables/Results/{} - Reference metrics per assembler.csv".format(distribution))
    print(distribution)
    display(ref_stats_per_reference_per_assembler)

Even


,Algorithm,L90,LSA,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,,
ABySS,Multiple k-mer De Bruijn graph,60.88 [0;1004],0.32 [0;0],110190.23 [0;351502],110089.38 [0;351502],105.21 [0;2170],Metagenomic,2536099.3 [0;6780808],0.92 [0;0],97.05 [0;1404],0.99 [0;1],0.95 [0;1],0.37 [0;8],0.65 [0;14],1.0 [0;1],1.0 [0;1],0.99 [0;1]
BCALM2,Single k-mer De Bruijn graph,51.2 [0;921],0.04 [0;0],6932.33 [0;65138],6539.94 [0;64934],0.0 [0;0],Genomic,1236061.67 [0;6510588],0.37 [0;0],272.97 [0;1495],0.5 [0;1],0.5 [0;1],0.0 [0;0],0.0 [0;0],0.5 [0;1],0.5 [0;1],0.5 [0;1]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,31.33 [0;112],0.35 [0;1],106093.36 [2176;452364],96782.33 [2176;351186],0.0 [0;0],Metagenomic,2523831.42 [2176;6716518],0.98 [0;1],59.92 [1;199],1.0 [0;1],0.98 [0;1],0.14 [0;3],0.25 [0;6],1.0 [0;1],1.0 [0;1],1.0 [0;1]
IDBA-UD,Multiple k-mer De Bruijn graph,68.82 [0;283],0.27 [0;1],55837.36 [0;254629],54555.43 [0;254629],0.0 [0;0],Metagenomic,2507606.06 [0;6710104],0.94 [0;1],121.11 [0;449],0.97 [0;1],0.87 [0;1],1.12 [0;9],1.94 [0;15],0.98 [0;1],0.98 [0;1],0.97 [0;1]
MEGAHIT,Multiple k-mer De Bruijn graph,18.31 [0;55],0.37 [0;1],138989.33 [2215;542872],141793.83 [2215;542872],0.0 [0;0],Metagenomic,2534425.75 [2215;6737986],0.99 [0;1],44.39 [1;141],1.0 [0;1],0.93 [0;1],0.97 [0;4],1.99 [0;7],1.0 [0;1],1.01 [0;1],1.0 [0;1]
MetaHipMer2,Multiple k-mer De Bruijn graph,21.69 [0;110],0.23 [0;0],58464.84 [0;257424],57747.71 [0;257424],0.0 [0;0],Metagenomic,1722416.51 [0;6717767],0.68 [0;0],74.19 [0;361],0.94 [0;1],0.94 [0;1],0.0 [0;0],0.0 [0;0],0.94 [0;1],0.94 [0;1],0.94 [0;1]
metaSPAdes,Multiple k-mer De Bruijn graph,20.14 [0;90],0.34 [0;1],133559.97 [1677;475510],132927.86 [1677;475510],0.0 [0;0],Metagenomic,2530047.11 [2076;6726884],0.98 [0;1],39.33 [1;167],1.0 [0;1],0.99 [0;1],0.14 [0;2],0.33 [0;6],1.01 [0;1],1.01 [0;1],1.0 [0;1]
MINIA,Single k-mer De Bruijn graph,83.4 [0;406],0.08 [0;0],17273.69 [0;65138],16631.06 [0;64960],0.0 [0;0],Genomic,2345231.06 [0;6518607],0.74 [0;0],307.75 [0;1146],0.81 [0;1],0.79 [0;1],0.0 [0;0],0.0 [0;0],0.81 [0;1],0.81 [0;1],0.81 [0;1]
SKESA,Multiple k-mer De Bruijn graph,57.92 [0;670],0.27 [0;0],76053.75 [1404;351014],75134.92 [0;351014],0.0 [0;0],Genomic,2496709.31 [1460;6673601],0.9 [0;0],96.06 [1;933],1.0 [0;1],0.87 [0;1],0.67 [0;7],1.17 [0;15],1.0 [0;1],1.0 [0;1],1.0 [0;1]


### Genomes vs Plasmids

In [91]:
ref_stats_per_replicon_type = pd.DataFrame()
# Genome
row={'Replicon': "Genome"}
lala=reference_pipeline_metrics_df[~reference_pipeline_metrics_df['reference'].str.contains("plasmid")].describe()
for column in lala.columns:
    mean = lala.loc['mean',column]
    std = lala.loc['std',column]
    row[column] = "{}(± {})".format(round(mean, 2), round(std, 2))
ref_stats_per_replicon_type =ref_stats_per_replicon_type .append(row, ignore_index=True)

# Plasmid
row={'Replicon': "Plasmid"}
lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df['reference'].str.contains("plasmid")].describe()
for column in lala.columns:
    mean = lala.loc['mean',column]
    minimum = lala.loc['min',column]
    maximum = lala.loc['max',column]
    row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    
ref_stats_per_replicon_type =ref_stats_per_replicon_type .append(row, ignore_index=True)
ref_stats_per_replicon_type  = ref_stats_per_replicon_type .set_index('Replicon')
ref_stats_per_replicon_type.to_csv("Tables/Results/Reference metrics per replicon type.csv")
display(ref_stats_per_replicon_type)

,L90,LSA,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,identity,lowest_identity,misassembled contigs,misassembly events,multiplicity,parsimony,validity
Replicon,,,,,,,,,,,,,,,
Genome,59.22(± 105.72),0.09(± 0.08),122637.65(± 120855.79),121330.29(± 120588.4),74.15(± 816.88),3362222.41(± 1746712.85),0.87(± 0.28),168.33(± 281.98),0.94(± 0.24),0.89(± 0.26),0.53(± 1.45),0.97(± 2.63),0.94(± 0.24),0.94(± 0.24),0.94(± 0.24)
Plasmid,0.85 [0;18],0.58 [0;1],6502.86 [0;55639],6365.95 [0;55639],9.49 [0;835],22670.82 [0;109516],0.72 [0;1],2.91 [0;33],0.78 [0;1],0.77 [0;1],0.0 [0;0],0.0 [0;0],0.78 [0;1],0.79 [0;1],0.78 [0;1]
